In [1]:
import os
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

In [2]:
from IPython.display import Audio, display

url_exception = "http://www.wav-sounds.com/movie/austinpowers.wav"
url_exception = "https://www.myinstants.com/media/sounds/roblox-death-sound_1.mp3"


def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(url=url_exception, autoplay=True))


get_ipython().set_custom_exc((Exception,), play_sound)

url_done = (
    "https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav"
)
url_done = "https://www.myinstants.com/media/sounds/taco-bell-bong-sfx.mp3"
url_done = "https://www.myinstants.com/media/sounds/magic_immune.mp3"
# url_done="https://www.myinstants.com/media/sounds/tindeck_1.mp3"
# url_done="https://www.myinstants.com/media/sounds/dun_dun_1.mp3"


def allDone():
    display(
        Audio(
            url=url_done,
            autoplay=True,
        )
    )

In [3]:
data_dir = os.path.join(os.getcwd(), "data")
drone_dir = os.path.join(data_dir, "tir_pavel")

In [4]:
video0 = np.load(os.path.join(drone_dir, "pavel0.npy"))
print(video0.shape)

(1144, 436, 496)


In [5]:
import datetime as dt

timestamp0 = np.load(os.path.join(drone_dir, "timestamp0.npy")).squeeze()
print(video0.shape)


def md(matlab_datenum):
    day = dt.datetime.fromordinal(int(matlab_datenum))
    dayfrac = dt.timedelta(days=matlab_datenum % 1) - dt.timedelta(days=366)
    return day + dayfrac


matlab_to_date = np.vectorize(md)
timestamp0 = matlab_to_date(timestamp0)

(1144, 436, 496)


In [21]:
video0_df = pd.DataFrame({"frame": [*video0]})
video0_df.index = timestamp0
video0_df

,frame
2019-08-06 12:29:34.000004,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:35.000008,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:36.000003,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:37.000007,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:38.000002,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
...,...
2019-08-06 12:48:33.000001,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:34.000006,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:35.000001,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:36.000005,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."


#### merging the two:

making the data more coarse: 
* https://stackoverflow.com/a/39952846/6293070
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
* we probably want to resample down to half seconds? and take that average?

merging the two:
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html#pandas.merge_asof
* https://stackoverflow.com/a/43979285/6293070
    

In [32]:
label_df_1 = pd.read_hdf(os.path.join(drone_dir, "pavel_label_1"), "/df")
label_df_2 = pd.read_hdf(os.path.join(drone_dir, "pavel_label_2"), "/df")
label_df = pd.concat([label_df_1, label_df_2], axis=0)
print(label_df.shape)

(503881, 29)


In [37]:
label_df = label_df.resample("1S").median()
print(label_df.shape)

(1924190, 29)


In [38]:
video0_df_w_lables = pd.merge_asof(
    video0_df,
    label_df,
    left_index=True,
    right_index=True,
    tolerance=pd.Timedelta("5S"),
    direction='nearest'
)

# print(video0_df.head(10))
print(video0_df_w_lables.head(10))

                                                                        frame  \
2019-08-06 12:29:34.000004  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:35.000008  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:36.000003  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:37.000007  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:38.000002  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:39.000007  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:40.000001  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:41.000006  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:42.000000  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:43.000005  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   

                                   0         1        2         3         4  \
2019-08-06 12:29:34.000004  1

Somehow take the center of this:

for now just take width and height from shape
divide it by four

then take the middle half

In [18]:
# might not be the right order!
frames, height, width = video0.shape

cut_off = 4

left = width // cut_off
right = width - left
top = height // cut_off
bottom = height - top

video0 = video0[:, top:bottom, left:right]
video0.shape

(1144, 218, 248)

label = np.load(os.path.join(drone_dir, "pavel0.npy"))